In [225]:
import pandas as pd
df_products_translated=pd.read_csv('./data/brazilian_e-commerce/olist_products_dataset_translated.csv')
df_products_translated.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumes
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials


In [226]:
df_products_translated['product_category_name_english'].value_counts()

product_category_name_english
Bed and Bathroom Products             3029
Sports and Leisure                    2867
Furniture and Decoration              2657
Beauty and Health Products            2444
Household Essentials                  2335
                                      ... 
Home Comfort (2)                         5
Children's and Teenagers' Clothing       5
PC Gamer                                 3
Insurance and Services                   2
Music CDs and DVDs                       1
Name: count, Length: 72, dtype: int64

In [227]:
df_products_translated['product_category_name_english'].unique()

array(['Perfumes', 'Arts', 'Sports and Leisure', 'Baby Products',
       'Household Essentials', 'Musical Instruments', 'Cool Stuff',
       'Furniture and Decoration', 'Electrical Appliances', 'Toys',
       'Bed and Bathroom Products',
       'Construction Tools and Safety Equipment', 'Computer Accessories',
       'Beauty and Health Products', 'Luggage and Travel Accessories',
       'Garden Tools', 'Office Furniture', 'Automotive Products',
       'Electronics', 'Footwear and Fashion', 'Telephony',
       'Paper Products', 'Handbags and Fashion Accessories',
       'Personal Care Products', 'Home Construction',
       'Watches and Gift Items', 'Construction Tools for Construction',
       'Pet Shop', 'Electrical Portables',
       'Agriculture, Industry, and Commerce', nan,
       'Living Room Furniture', 'Signage and Security Equipment',
       'Climate Control', 'Consoles and Games',
       'Books and General Interest Products',
       'Underwear and Beach Fashion', "Men's Clothi

In [228]:
import sys
print(sys.executable)


/opt/miniconda3/bin/python


In [229]:
import sys
!{sys.executable} -m pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [235]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

categories = df_products_translated['product_category_name_english'].unique().tolist()
embeddings = model.encode(categories)
X = np.array(embeddings)
print(f"Embeddings shape: {X.shape}")  # (72, embedding_dim)


Embeddings shape: (73, 384)


In [236]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

scores = {}
for k in range(2, 20):
    kmeans_tmp = KMeans(n_clusters=k, random_state=42, n_init=20).fit(X)
    score = silhouette_score(X, kmeans_tmp.labels_)
    scores[k] = score
    print(f"k={k}, silhouette score={score:.3f}")

best_k = max(scores, key=scores.get)
print(f"Optimal number of clusters: {best_k}")


k=2, silhouette score=0.046
k=3, silhouette score=0.055
k=4, silhouette score=0.049
k=5, silhouette score=0.061
k=6, silhouette score=0.073
k=7, silhouette score=0.065
k=8, silhouette score=0.083
k=9, silhouette score=0.089
k=10, silhouette score=0.077
k=11, silhouette score=0.100
k=12, silhouette score=0.116
k=13, silhouette score=0.125
k=14, silhouette score=0.129
k=15, silhouette score=0.118
k=16, silhouette score=0.127
k=17, silhouette score=0.136
k=18, silhouette score=0.137
k=19, silhouette score=0.151
Optimal number of clusters: 19


In [237]:
from sklearn.cluster import KMeans

# Fit KMeans with n_init=20 to prevent empty clusters
kmeans = KMeans(
    n_clusters=best_k,
    random_state=42,
    n_init=20
).fit(X)

labels = kmeans.labels_

# Map each category to its numeric cluster
category_map = {
    cat: f"Cluster {label}" 
    for cat, label in zip(categories, labels)
}

# Add numeric cluster column to the DataFrame
df_products_translated["compressed_category"] = (
    df_products_translated["product_category_name_english"]
    .map(category_map)
)


In [238]:
df_products_translated

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,compressed_category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumes,Cluster 4
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts,Cluster 17
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure,Cluster 13
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products,Cluster 4
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials,Cluster 6
...,...,...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,Furniture and Decoration,Cluster 9
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,Building Tools and Lighting,Cluster 2
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,Bed and Bathroom Products,Cluster 4
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,Computer Accessories,Cluster 18


In [239]:
import requests, json

def ollama_chat(prompt, model_name="llama3:8b"):
    url = "http://localhost:11434/api/chat"
    payload = {"model": model_name, "messages": [{"role": "user", "content": prompt}]}
    response = requests.post(url, json=payload, stream=True)
    full_output = ""
    for line in response.iter_lines():
        if not line: 
            continue
        try:
            data = json.loads(line.decode("utf-8"))
        except json.JSONDecodeError:
            continue
        if "message" in data and "content" in data["message"]:
            full_output += data["message"]["content"]
    return full_output.strip()


In [240]:
# Generate descriptive names for each numeric cluster
cluster_names = {}

for i in range(best_k):
    items = [cat for cat, label in category_map.items() if label == f"Cluster {i}"]
    prompt = (
        f"Cluster {i}: Name a meaningful category for these items:\n{items}\n"
        "Return only a single short descriptive name without quotes or punctuation."
    )
    cluster_names[f"Cluster {i}"] = ollama_chat(prompt)


In [241]:
seen = set()
unique_names = {}

for k, name in cluster_names.items():
    if name in seen:
        name = f"{name} ({k})"  # append cluster number if duplicate
    seen.add(name)
    unique_names[k] = name


In [242]:
df_products_translated["compressed_category"] = (
    df_products_translated["compressed_category"]
    .map(unique_names)
)


In [243]:
df_products_translated['compressed_category'].unique()

array(['Household Essentials', 'Creative Pursuits', 'Social Gatherings',
       'Home Goods', 'Sound', 'Giftware', 'Household Furnishings',
       'Gifts', 'Building Essentials', 'Clothing', 'Phone Services',
       'Economy', 'Electronics', 'HVAC', 'Video Gaming', 'Publishing',
       'Living Spaces', 'Digital Imaging', 'Home Entertainment'],
      dtype=object)

In [244]:
df_products_translated['compressed_category'].nunique()

19

In [245]:
# Check how many numeric clusters exist
print(len(category_map.values()), len(set(category_map.values())))

# Check cluster name counts
from collections import Counter
Counter(df_products_translated['compressed_category'])


73 19


Counter({'Household Essentials': 10065,
         'Giftware': 5048,
         'Home Goods': 3584,
         'Household Furnishings': 3271,
         'Social Gatherings': 3170,
         'Gifts': 2195,
         'Building Essentials': 1674,
         'Phone Services': 1250,
         'Electronics': 610,
         'Sound': 374,
         'Clothing': 372,
         'Publishing': 370,
         'Video Gaming': 320,
         'Economy': 248,
         'HVAC': 124,
         'Living Spaces': 116,
         'Creative Pursuits': 74,
         'Home Entertainment': 49,
         'Digital Imaging': 37})

In [246]:
df_products_translated.to_csv("products_with_compressed_categories.csv", index=False)


In [247]:
df_products_translated

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,compressed_category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumes,Household Essentials
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts,Creative Pursuits
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure,Social Gatherings
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products,Household Essentials
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials,Home Goods
...,...,...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,Furniture and Decoration,Household Furnishings
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,Building Tools and Lighting,Building Essentials
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,Bed and Bathroom Products,Household Essentials
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,Computer Accessories,Giftware


In [248]:
mapping = df_products_translated[['product_category_name_english', 'compressed_category']].drop_duplicates()
print(mapping.sort_values('compressed_category'))


            product_category_name_english  compressed_category
31                           Garden Tools  Building Essentials
73    Construction Tools for Construction  Building Essentials
70                      Home Construction  Building Essentials
271           Building Tools and Lighting  Building Essentials
498             Garden and Building Tools  Building Essentials
...                                   ...                  ...
5                     Musical Instruments                Sound
652                                 Music                Sound
591                                 Audio                Sound
134                    Consoles and Games         Video Gaming
1628                             PC Gamer         Video Gaming

[73 rows x 2 columns]


In [249]:
import pandas as pd

# Group original categories by compressed category
category_mapping_table = (
    df_products_translated
    .groupby('compressed_category')['product_category_name_english']
    .apply(list)
    .reset_index()
)

# Optional: sort by compressed category
category_mapping_table = category_mapping_table.sort_values('compressed_category')

# Display the table
print(category_mapping_table)


      compressed_category                      product_category_name_english
0     Building Essentials  [Construction Tools and Safety Equipment, Gard...
1                Clothing  [Footwear and Fashion, Underwear and Beach Fas...
2       Creative Pursuits  [Arts, Arts, Arts, Arts, Arts and Handicrafts,...
3         Digital Imaging  [Tablets and Image Printing, Film and Photogra...
4                 Economy  [Agriculture, Industry, and Commerce, Industry...
5             Electronics  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
6                   Gifts  [Toys, Toys, Toys, Toys, Pet Shop, Pet Shop, T...
7                Giftware  [Cool Stuff, Computer Accessories, Luggage and...
8                    HVAC  [Climate Control, Climate Control, Climate Con...
9      Home Entertainment  [DVDs and Blu-rays, DVDs and Blu-rays, DVDs an...
10             Home Goods  [Household Essentials, Electrical Appliances, ...
11   Household Essentials  [Perfumes, Baby Products, Bed and Bathroom Pro...